<a href="https://colab.research.google.com/github/cxbxmxcx/PAIGCP/blob/master/PAIGCP_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time


In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
path_to_file

2646016/2638744 [==============================] - 0s 0us/step


'/root/.keras/datasets/spa-eng/spa.txt'

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(path_to_file, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

lines[:10]

['Go.\tVe.',
 'Go.\tVete.',
 'Go.\tVaya.',
 'Go.\tVáyase.',
 'Hi.\tHola.',
 'Run!\t¡Corre!',
 'Run.\tCorred.',
 'Who?\t¿Quién?',
 'Fire!\t¡Fuego!',
 'Fire!\t¡Incendio!']

In [ ]:
num_samples = 1000
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
num_encoder_tokens = len(input_characters)
num_encoder_tokens, ",".join(input_characters)

(61,
 " ,!,',,,.,0,1,3,8,9,:,?,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,R,S,T,U,V,W,Y,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z")

In [ ]:
target_characters = sorted(list(target_characters))
num_decoder_tokens = len(target_characters)
num_decoder_tokens, ",".join(target_characters)

(70,
 '\t,\n, ,!,,,.,0,3,8,:,?,A,B,C,D,E,F,G,H,I,J,L,M,N,O,P,Q,R,S,T,U,V,Y,a,b,c,d,e,f,g,h,i,j,l,m,n,o,p,q,r,s,t,u,v,x,y,z,¡,¿,Á,É,Ó,Ú,á,é,í,ñ,ó,ú,ü')

In [ ]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
print('Max sequence length for inputs:', max_encoder_seq_length)

Max sequence length for inputs: 11


In [ ]:
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Max sequence length for outputs:', max_decoder_seq_length)

Max sequence length for outputs: 31


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Reshape

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
#encoder_embedded = Embedding(input_dim=num_encoder_tokens, output_dim=64)(encoder_inputs)
#encoder_reshape = Reshape((-1, 64))(encoder_embedded)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
batch_size = 64
epochs = 100
encoder_input_data.shape,decoder_input_data.shape

((1000, 11, 61), (1000, 31, 70))

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/100
13/13 [==============================] - 1s 76ms/step - loss: 3.3758 - accuracy: 0.5017 - val_loss: 2.1734 - val_accuracy: 0.5777
Epoch 2/100
13/13 [==============================] - 0s 19ms/step - loss: 1.7953 - accuracy: 0.6275 - val_loss: 1.9087 - val_accuracy: 0.5744
Epoch 3/100
13/13 [==============================] - 0s 19ms/step - loss: 1.6027 - accuracy: 0.6265 - val_loss: 1.7623 - val_accuracy: 0.5731
Epoch 4/100
13/13 [==============================] - 0s 20ms/step - loss: 1.5276 - accuracy: 0.6279 - val_loss: 1.7184 - val_accuracy: 0.5808
Epoch 5/100
13/13 [==============================] - 0s 19ms/step - loss: 1.4907 - accuracy: 0.6279 - val_loss: 1.6831 - val_accuracy: 0.5787
Epoch 6/100
13/13 [==============================] - 0s 19ms/step - loss: 1.4550 - accuracy: 0.6304 - val_loss: 1.7033 - val_accuracy: 0.5885
Epoch 7/100
13/13 [==============================] - 0s 19ms/step - loss: 1.4257 - accuracy: 0.6348 - val_loss: 1.6383 - val_accuracy: 0.5852
Epoch 

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Go.
Decoded sentence: Vete.

-
Input sentence: Hi.
Decoded sentence: Vente.

-
Input sentence: Run!
Decoded sentence: ¡Ve elo!

-
Input sentence: Run.
Decoded sentence: Vete.

-
Input sentence: Who?
Decoded sentence: ¿Quién anor?

-
Input sentence: Fire!
Decoded sentence: ¡Despara!

-
Input sentence: Fire!
Decoded sentence: ¡Despara!

-
Input sentence: Fire!
Decoded sentence: ¡Despara!

-
Input sentence: Help!
Decoded sentence: ¡Vete!

-
Input sentence: Help!
Decoded sentence: ¡Vete!

-
Input sentence: Help!
Decoded sentence: ¡Vete!

-
Input sentence: Jump!
Decoded sentence: ¡Dete alo!

-
Input sentence: Jump.
Decoded sentence: Satente.

-
Input sentence: Stop!
Decoded sentence: ¡Vete a!

-
Input sentence: Stop!
Decoded sentence: ¡Vete a!

-
Input sentence: Stop!
Decoded sentence: ¡Vete a!

-
Input sentence: Wait!
Decoded sentenc